In [8]:
setwd('C:/Users/Mhuth/Desktop/PCRPVA') #only for local computations
files <- c('PCA_PropVar', 'PCA', 'PCR_cv', 'PCR_predict', 'PCR', 'random_discreteVariables', 'random_VCV') #names of files to read
for (i in 1:length(files)) { #loop to read all files
    source(paste('R/', files[i], '.R', sep = '')) 
}


---
# 3. Model
---

## 3.1 Structural Model

### 3.1.1 Causal Graph

Use R-package for that

### 3.1.2 Equations to build the variables

**Parents' Education**

Since count data, assumed to follow a count distribution. 
- Chose Generalized Poisson, to hold it general -> equi- and overdispersion is feasible ((Consul, Jain) (1973), A generalization of the Poissin distribution) -> show distribution
- as in causal graph, assumed to be exogenous
if we have data,we can set the true parameters by setting the true variance and mean by:
- mean: $\frac{\lambda}{1-\eta}$
- variance: $\frac{\lambda}{(1-\eta)^3}$ -> compute variance first $\eta = 1 - \frac{\sqrt{mean}}{std}$ -> lambda is $mean*(1-\eta)$  

**Ability**

$ability \sim \mathcal{N}(0, \sigma^2_{ability})$, independent -> see causal graph, from The effect of schooling and ability on achievement test scores [(Hansen, Heckman, Mullen)](https://www.sciencedirect.com/science/article/pii/S0304407603002598)

**Test Scores**

Standardized test in the unit interval $[0,1]$. Can be adjusted such that it fits the data. Default is uniform distributed. 

latent process $T^*_{i, g, s}$, $g \in \{7,11 \}$, $s \in \{ \text{math}, \text{reading}\}$, $i = 1, \dots, n$ 

extend that errors are uncorrelated and standard normal

*Latent Test Scores in 7th Grade*
$$T^*_{i,7,s} = \gamma_{ability} \cdot ability + \varepsilon_{i,7,s} \sim \mathcal{N}(0, \sigma^2_{ability} + 1)$$, 

equation from: The effect of schooling and ability on achievement test scores (Hansen, Heckman, Mullen) (equation 29, slightly modified). They only round: mine is better because its more flexible and can be modified to meet any taste

*Latent Test Scores in 11th Grade*
$$T^*_{i,11,s} = T^*_{i,7,s} + \varepsilon_{i,11,s} \sim \mathcal{N}(0, \sigma^2_{ability} + 2)$$

with respective distribution functions $\phi_{g}$ (same for reading and math, only more variance in 11th Grade). 

*Test Scores*
$$T_{i,7,s} = \phi_g \left( T^*_{i,7,s} \right) \sim unif(0,1)$$ 
(shown in appendix)

*Optional transforming*

categorial or basically any desired distribution (in style of a random number generator), if yes, add to name of T an index that shows it is the new latent process.

**Number of Siblings**

- decrease since demand for children decreases (since opportunity costs increase)
- The Effect of Education on Fertility: Evidence from a Compulsory Schooling Reform (Kamila Cygan-Rehm and Miriam Maeder)
'More specifically, one additional year of schooling ultimately reducesthe number of children by more than 0.1, and increases the probability of childlessness by about 2-5 percentage points'
- m is maximal number of education until the 0.1 decrease it has an influence
- E_i is years of schooling of parents (need to change in equations. former e_i$

$$N_i = \sum_{j = 1}^{m-e_i} B_{ij} + W_i $$

$B_j \sim \mathcal{B}(p)$ are all Bernoulli distributed and $W_i \sim GP(\lambda^{sib}, \eta^{sib})$ are all generalized Poisson distributed. All $B_{ij}$ and $W_i$ are independent. Note that $\sum_{j = 1}^{m-e_i} B_{ij} | e_i \sim Bin(m-e_i, p)$. Show that the difference of one year more schooling is -p

$$\mu_N = \text{E}(N_i) = \text{E}\left(\sum_{j = 1}^{m-e_i} B_{i,j} + W_i \right) =  \text{E}\left[ \text{E}\left( \sum_{j = 1}^{m-e_i} B_{ij} | e_i \right)\right] + \frac{\lambda^{sib}}{1-\eta^{sib}}  = \text{E}\left[ (m - e_i) \cdot p \right] + \frac{\lambda^{sib}}{1-\eta^{sib}} = (m - \mu_{s}) \cdot p + \frac{\lambda^{sib}}{1-\eta^{sib}}$$
using variance decomposition of formula *()* for the second equality sign
$$\sigma^2_N = \text{Var}(N_i) = \text{Var}\left(\sum_{j = 1}^{m-e_i} B_{i,j}\right) + \text{Var}(W_i) =  \text{E}\left[ \text{Var}\left(\sum_{j = 1}^{m-e_i} B_{i,j}|e_i\right) \right] + \text{Var}\left[\text{E}\left(\sum_{j = 1}^{m-e_i} B_{i,j}|e_i) \right) \right] + \frac{\lambda^{sib}}{(1-\eta^{sib})^3} = \text{E}[(m-e_i)\cdot p \cdot (1-p)] + \text{Var}[(m-e_i)\cdot p] + \frac{\lambda^{sib}}{(1-\eta^{sib})^3} = (m-\mu_s)\cdot p \cdot (1-p) + p^2 \cdot \sigma^2_s + \frac{\lambda^{sib}}{(1-\eta^{sib})^3}$$

from the equations it follows (move to appendix)
\begin{align}
 \frac{\lambda^{sib}}{1-\eta^{sib}} =& \mu_N - (m - \mu_{s}) \cdot p \\
 \frac{\lambda^{sib}}{(1-\eta^{sib})^3} =& \sigma^2_N - (m-\mu_s)\cdot p \cdot (1-p) + p^2 \cdot \sigma^2_s 
\end{align}

\begin{align}
 \Longleftrightarrow & \frac{\mu_N - (m - \mu_{s}) \cdot p}{(1-\eta^{sib})^2} = \sigma^2_N - (m-\mu_s)\cdot p \cdot (1-p) + p^2 \cdot \sigma^2_s \\
 \Longleftrightarrow & \frac{\mu_N - (m - \mu_{s}) \cdot p}{\sigma^2_N - (m-\mu_s)\cdot p \cdot (1-p) + p^2 \cdot \sigma^2_s} =  (1-\eta^{sib})^2\\
 \Longleftrightarrow & \eta^{sib}=  1 - \left(\frac{\mu_N - (m - \mu_{s}) \cdot p}{\sigma^2_N - (m-\mu_s)\cdot p \cdot (1-p) + p^2 \cdot \sigma^2_s}\right)^{0.5} 
\end{align}

means for lambda

$$
\lambda^{sib} = [\mu_N - (m - \mu_{s}) \cdot p] \cdot (1-\eta^{sib})
$$


**Years of schooling**

write that it is common in the count literature to put covariates (ability) on lambda

I assume that the years of schooling $s_i$ of an individual $i$ is a function of the parent's years of education and an individual's ability. Additionally, the years of education again follow a count distribution. I assume, similar to the number of siblings, that the expected value is proportional to parent's education. Moreover, there is an additive error term that is dependent on ability. Hence,

\begin{align}
    s_i = \sum_{j = 1}^{e_i} B_{i,j}^{sch} + W^{sch}_i
\end{align}

$W^{sch}_i |a_i \sim GP(\lambda_i^\text{sch}, \eta^{sch})$ and $\lambda_i = k \cdot \phi(a_i) \sim unif[0, k]$, whereby $k \in \mathbb{R}$. It is well-known that $\text{E}(\lambda_i) = \frac{k}{2}$ and $\text{Var}(\lambda_i) = \frac{k^2}{12}$. $B_{i,j}^{sch} \sim \mathcal{B}(q)$.
It is common in the count literature to put the ability on the error. Using the law of iterated expectations and the variance decomposition formula *()* it follows that

\begin{align}
\text{E}(W^{sch}_i) =& \text{E}\left[\text{E}\left(W^{sch}_i|a_i \right )\right] = \text{E}\left(\frac{\lambda_i}{1-\eta}\right) = \frac{1}{1-\eta}\text{E}\left[k \cdot \phi(a_i) \right] = \frac{k}{2(1-\eta)} \\
\text{Var}(W^{sch}_i) =& \text{E}\left[\text{Var}\left( W^{sch}_i | a_i \right) \right] + \text{Var}\left[\text{E}\left( W^{sch}_i | a_i \right) \right] = \text{E}\left( \frac{\lambda_i}{(1-\eta)^3} \right) + \text{Var}\left( \frac{\lambda_i}{1-\eta} \right) = \frac{k}{2(1-\eta)^3} + \frac{k^2}{12(1-\eta)^2}
\end{align}

with the above formulas and what was derived for the number of siblings. Second step of variance equality sign I make use that ability and parent's years of education are uncorrelated.

\begin{align}
    \mu_{sch} =& \text{E}(s_i) = \text{E} \left( \sum_{j = 1}^{e_i} B_{i,j}^{sch} + W^{sch}_i \right) =  \text{E} \left( \sum_{j = 1}^{e_i} B_{i,j}^{sch} \right) + \text{E} \left( W^{sch}_i \right) = \mu_{PEduc} \cdot q + \frac{k}{2(1-\eta)} \\
    \sigma^2_{sch} =& \text{Var}(s_i) = \text{Var} \left( \sum_{j = 1}^{e_i} B_{i,j}^{sch} + W^{sch}_i \right) = \text{Var} \left( \sum_{j = 1}^{e_i} B_{i,j}^{sch} \right)  + \text{Var} \left( W^{sch}_i \right) = \mu_{PEduc} \cdot q \cdot (1-q) + \sigma_{PEduc}^2 \cdot q + \frac{k}{2(1-\eta)^3} + \frac{k^2}{12(1-\eta)^2}
\end{align}

from the first equation it follows
\begin{align}
    \frac{1}{1-\eta} = \frac{2(\mu_{sch} - \mu_{PEduc}  \cdot q) }{k}
\end{align}

yielding for the second equation that

\begin{align}
    \sigma^2_{sch} = \mu_{PEduc} \cdot q \cdot (1-q) + \sigma_{PEduc}^2 \cdot q + \frac{k}{2} \cdot \left( \frac{2(\mu_{sch} - \mu_{PEduc}  \cdot q) }{k}\right)^3 + \frac{k^2}{12} \cdot \left( \frac{2(\mu_{sch} - \mu_{PEduc} \cdot q)  }{k}\right)^2 \\
\Longleftrightarrow \sigma^2_{sch} -  \mu_{PEduc} \cdot q \cdot (1-q) - \sigma_{PEduc}^2 \cdot q - \frac{1}{3} (\mu_{sch} - \mu_{PEduc} \cdot q)^2 = 4 \frac{(\mu_{sch} - \mu_{PEduc}  \cdot q)^2 }{k^2} \\
\Longleftrightarrow k^2 = \frac{4(\mu_{sch} - \mu_{PEduc}  \cdot q)^2 }{\sigma^2_{sch} -  \mu_{PEduc} \cdot q \cdot (1-q) - \sigma_{PEduc}^2 \cdot q - \frac{1}{3} (\mu_{sch} - \mu_{PEduc} \cdot q)^2} \\
\Longleftrightarrow k = \pm \frac{2(\mu_{sch} - \mu_{PEduc}  \cdot q) }{\left[\sigma^2_{sch} - \mu_{PEduc} \cdot q \cdot (1-q) - \sigma_{PEduc}^2 \cdot q - \frac{1}{3} (\mu_{sch} - \mu_{PEduc} \cdot q)^2 \right]^{0.5}} 
\end{align}

### 3.1.3 Covariances

---
Model of Card (2001) with standard mincer equation

\begin{align}
\ln (Y_i) = a_0 + a_i + \beta_1 \cdot S_i - \frac{1}{2} k_1 S_i^2
\tag{3.1}
\end{align}

I extend this by adding work experience $w$ as a second order polynomial and an additional vector of covariates $A_i$. The equation to be estimated is therefore

\begin{align}
\ln (Y_i) = a_0 + a_i + \beta_1 S_i + \beta_2 S_i^2 + \beta_3 w_i + \beta_4 w_i^2 + A_i \gamma + \varepsilon_i
\tag{3.1}
\end{align}

The distribution of most covariates is derived from the descriptives in Blundell et al (2005). Since I wanna compare the results of the true VCV matrix with the estimated VCV matrices, my approach is to set up a system of equations, such that I have full control about the correlation structure of the simulated data. This is not straightforward since some variables, like years of schooling, follow a discrete distribution and hence, they cannot be implemented via an R function directly.